In [ ]:
pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.9 MB/s eta 0:00:00


In [3]:
import json
import re
from transformers import BertTokenizer, BertForQuestionAnswering
import torch

# Initialize the BERT model and tokenizer
model = BertForQuestionAnswering.from_pretrained('KB/bert-base-swedish-cased')
tokenizer = BertTokenizer.from_pretrained('KB/bert-base-swedish-cased')

def extract_text_content(text_path, encodings=['utf-8', 'ISO-8859-1', 'Windows-1252']):
    for encoding in encodings:
        try:
            with open(text_path, 'r', encoding=encoding) as file:
                all_text = file.read()
            return all_text
        except UnicodeDecodeError:
            continue
    raise ValueError("Unable to decode the text using any of the specified encodings: {}".format(encodings))


def get_context_sentences(text, query, window_size=1):
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)
    query_sentences = []

    for idx, sentence in enumerate(sentences):
        if query.lower() in sentence.lower():
            start_idx = max(0, idx - window_size)
            end_idx = min(len(sentences), idx + window_size + 1)
            context = ' '.join(sentences[start_idx:end_idx])
            query_sentences.append(context)

            # Retrieve words before each new line
            lines = re.findall(r'(.+?)(?:\n|$)', context, re.DOTALL)
            for line in lines[:-1]:  # Exclude the last line
                query_sentences.append(line.strip())

    return query_sentences

def generate_answer(user_question, text_content):
    context_sentences = get_context_sentences(text_content, user_question, window_size=1)
    answers = []

    for sentence in context_sentences:
        inputs = tokenizer.encode_plus(user_question, sentence, add_special_tokens=True, return_tensors="pt", max_length=512, truncation=True)

        # Get the output from the BERT model
        with torch.no_grad():
            outputs = model(**inputs)

        answer_start_scores = outputs.start_logits
        answer_end_scores = outputs.end_logits

        answer_start = torch.argmax(answer_start_scores)
        answer_end = torch.argmax(answer_end_scores) + 1

        answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))
        answers.append(answer)

    return ' '.join(answers)

def create_json(question, answer, context):
    return {
        "Fråga": question,
        "Svar": answer,
        "Kontext": context
    }

# Load existing data from output.json if it exists
try:
    with open('output.json', 'r', encoding='utf-8') as json_file:
        data = json.load(json_file)
except FileNotFoundError:
    data = []

text_sökväg = 'TDOK_2020-0216.pdf'  # Replace with your text file path
text_innehåll = extract_text_content(text_sökväg)

print("Hej! Jag är en chatbot som kan hjälpa dig med information från en textfil. Fråga mig vad som helst (eller skriv 'avsluta' för att sluta).")

while True:
    user_question = input("Ange din fråga: ")

    if user_question.lower() == 'avsluta':
        print("Tack för att du chattade med mig. Adjö!")
        break

    genererat_svar = generate_answer(user_question, text_innehåll)
    print("Svar:", genererat_svar)

    # Create JSON data (optional)
    json_data = create_json(user_question, genererat_svar, text_innehåll)
    data.append(json_data)

# Save data to output.json
with open('output.json', 'w', encoding='utf-8') as json_file:
    json.dump(data, json_file, ensure_ascii=False, indent=2)


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at KB/bert-base-swedish-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Hej! Jag är en chatbot som kan hjälpa dig med information från en textfil. Fråga mig vad som helst (eller skriv 'avsluta' för att sluta).
Ange din fråga: Vem bär ansvaret för att säkerställa att kompetenskravsspecifikationen för Lärare uppfylls?
Svar: 
Ange din fråga: Vem bär ansvaret för att säkerställa att kompetenskravsspecifikationen för Lärare uppfylls?
Svar: 
Ange din fråga: ansvaret för att säkerställa
Svar: 
Ange din fråga: kompetenskravsspecifikationen 
Svar: 


KeyboardInterrupt: Interrupted by user

In [7]:
import json
import re
from transformers import BertTokenizer, BertForQuestionAnswering
import torch

# Initialize the BERT model and tokenizer
model = BertForQuestionAnswering.from_pretrained('KB/bert-base-swedish-cased')
tokenizer = BertTokenizer.from_pretrained('KB/bert-base-swedish-cased')

def extract_text_content(text_path, encodings=['utf-8', 'ISO-8859-1', 'Windows-1252']):
    for encoding in encodings:
        try:
            with open(text_path, 'r', encoding=encoding) as file:
                all_text = file.read()
            return all_text
        except UnicodeDecodeError:
            continue
    raise ValueError("Unable to decode the text using any of the specified encodings: {}".format(encodings))


def generate_answer(user_question, text_content):
    inputs = tokenizer.encode_plus(user_question, text_content, add_special_tokens=True, return_tensors="pt", max_length=512, truncation='only_second')

    # Get the output from the BERT model
    with torch.no_grad():
        outputs = model(**inputs)

    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits

    answer_start = torch.argmax(answer_start_scores)
    answer_end = torch.argmax(answer_end_scores) + 1

    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))

    return answer


text_path = 'TDOK_2015-0430.pdf'  # Replace with your text file path
text_content = extract_text_content(text_path)

print("Hej! Jag är en chatbot som kan hjälpa dig med information från en textfil. Fråga mig vad som helst (eller skriv 'avsluta' för att sluta).")

while True:
    user_question = input("Ange din fråga: ")

    if user_question.lower() == 'avsluta':
        print("Tack för att du chattade med mig. Adjö!")
        break

    generated_answer = generate_answer(user_question, text_content)
    print("Svar:", generated_answer)


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at KB/bert-base-swedish-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Hej! Jag är en chatbot som kan hjälpa dig med information från en textfil. Fråga mig vad som helst (eller skriv 'avsluta' för att sluta).
Ange din fråga: TTJ
Svar: ##g ) / Creator ( Aspose Ltd . ) / Subject ( \ 376 \ 377 \ 000P \ 000r \ 000o \ 000v \ 000k \ 000 \ 366 \ 000r \ 000n \ 000i \ 000n \ 000g ) / Producer ( Aspose . Pdf for . NET 8 . 5 . 0 ) > > endobj 2 0 obj < < / OutputIntents [ < < / Info ( sRGB IEC61966 - 2 . 1
Ange din fråga: vad ar TTJ?
Svar: ##g ) / Creator ( Aspose Ltd . ) / Subject ( \ 376 \ 377 \ 000P \ 000r \ 000o \ 000v \ 000k \ 000 \ 366 \ 000r \ 000n \ 000i \ 000n \ 000g ) / Producer ( Aspose . Pdf for . NET 8 . 5 . 0 ) > > endobj 2 0 obj < < / OutputIntents [ < < / Info ( sRGB IEC61966 - 2 . 1


KeyboardInterrupt: Interrupted by user